# Páctica 2: Redes convolucionales

**Alumno**: Iván Cañaveral Sánchez

En esta práctica vamos a explorar e intentar comprender el funcionamiento de las redes convolutivas para el procesamiento de textos.

La práctica se estructura en las siguientes partes:

* **1. Comparativa con modelos revisados en la práctica anterior**, así como las técnicas de vectorización asociadas.
* **2. Exploración detallada** de este tipo de modelos, explorando cómo funcionan los filtros y cómo se generan los mapas de características, antes de comenzar a hacer pruebas con hiperparámetros.
* **3.** Finalmente, una vez que tenemos un entendimiento básico del funcionamiento interno estos modelos, llevaremos a cabo una serie de pruebas para ver cómo **impacta la modificación de ciertos hiperparámetros**.

## 0. Librerías y constantes

In [ ]:
import os
import re
import json
import time
import shutil
import string
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.manifold import TSNE

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

In [ ]:
sns.set(rc={'figure.figsize':(12,6)})

In [ ]:
import nltk
from nltk.stem import *
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
BATCH_SIZE = 32
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 64

## 1. Comparativa con modelos anteriores. Vectorización.

En esta sección vamos a comparar el funcionamiento de una red convolutiva simple con un perceptrón multicapa, prestando atención a los distintos métodos de vectorización.

Una diferencia importante en este punto es que los modelos vistos previamente requerían una representación que generase un único vector por cada texto o documento. De hecho, en la práctica anterior, cuando se utilizaban vectorizaciones a nivel de palabra, forzábamos la representación a nivel de texto utilizando una capa "flatten" en el modelo (también podrían haberse usado operaciones como medias, cálculo de centros, etc.)

En los modelos convolucinales, cada textos debe ser representado como una secuencia ordenada de palabras, para que estos modelos puedan detectar patrones dentro de las secuencias de texto.

### Carga y limpieza del texto

En este apartado vamos a descargar, limpiar y cargar los textos en un DataFrame de pandas, que dado que el tamaño no es muy grande, mantendremos cargado en memoria.

Dado que el proceso es el mismo que ne la práctica anterior, no entraremos en grandes detalles.

In [ ]:
!git clone https://github.com/ivanCanaveral/msc-datasets/

In [ ]:
raw_test_data = pd.read_csv('msc-datasets/movie-reviews/test_reviews.csv')
raw_train_data = pd.read_csv('msc-datasets/movie-reviews/train_reviews.csv')

In [ ]:
raw_test_data['partition'] = 'test'
raw_train_data['partition'] = 'train'
dataset = pd.concat([raw_test_data, raw_train_data])

In [ ]:
dataset = dataset.set_index(dataset.id).drop(columns=["id"])
dataset["length"] = dataset.review.str.split().apply(len)
dataset.head()

Revisamos rápidamente cuántos textos tenemos en cada una de las particiones y clases:

In [ ]:
dataset.groupby(by=["partition", "sentiment"]).count()

A continuación tenemos la clase que utilizamos para limpiar y procesar los textos. Ofrece 3 niveles distintos de procesado:

* **basic**: limpieza básica de stopwords, paso a minúsculas, etc.
* **lemma**: aplica lematización
* **stem**: aplica stemming

Los detalles de cada una de ellas se revisaron en la práctica anterior.

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, level='basic'):
        assert level in ['basic', 'lemma', 'stem'], "Wrong level value"
        self.level = level
        self.apply_lemma = level != 'basic'
        self.apply_stem = level == 'stem'
    
    def clean_text(self, text):
        letters_only = re.sub("[^a-zA-Z]", " ", text)
        words = letters_only.lower().split()
        stops = set(stopwords.words("english") + ['br'])
        words = [w for w in words if not w in stops]
        return words

    def lemmatize_words(self, words):
        wordnet_lemmatizer = WordNetLemmatizer()
        lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in words]
        return lemmatized
    
    def stem_words(self, words):
        stemmer = PorterStemmer()
        stemmed = [stemmer.stem(word) for word in words]
        return stemmed

    def parse_text(self, text):
        words = self.clean_text(text)
        if self.apply_lemma:
            words = self.lemmatize_words(words)
        if self.apply_stem:
            words = self.stem_words(words)
        return " ".join(words)

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return np.vectorize(self.parse_text)(X)

    def get_params(self, deep=True):
        return {
            "level": self.level
        }

Vamos a aplicar los tres tipos de preprocesado para poner disponer de ellos de cara a llevar a cabo distintas pruebas.

In [ ]:
for prep_type in ['basic', 'lemma', 'stem']:
  print("Procesando...", prep_type, end=' ')
  t0 = time.time()
  text_preprocessor = TextPreprocessor(prep_type)
  dataset[f'{prep_type}_review'] = text_preprocessor.transform(dataset.review)
  print(f"{time.time()-t0:02f} s.")

Generamos una etiqueta binaria para la variable "sentiment":

In [ ]:
dataset['label'] = dataset['sentiment'] == 'positive'
dataset['label'] = dataset['label'].astype('int32')

A continuación mostramos la estructura del dataset generado:

In [ ]:
dataset.head(2)

Para facilitar los desarrollos posteriores vamos a generar dos DataFrames separando el conjunto de datos de entrenamiento y el de evaluación (train y test respectivamente).

In [ ]:
train_df = dataset[dataset['partition'] == 'train']
test_df = dataset[dataset['partition'] == 'test']

### Vectorización

Por simplicidad, en adelante vamos a desarrollar los modelos en la librería `tensorflow`en la medida de lo posible, por lo que para facilitar los entrenamientos vamos a cargar los datos en un Dataset de tensorflow, que dispone de algunas ventajas frente al dataset de pandas que se había utilizado hasta ahora, tales como uno control sencillo de los batches.

In [ ]:
raw_train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df['lemma_review'], train_df['label'].values)).batch(BATCH_SIZE)
raw_test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df['lemma_review'], test_df['label'].values)).batch(BATCH_SIZE)

Exploramos cómo quedan los textos y las etiquetas dentro de estos datasets:

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(2):
    print("Review: ", text_batch.numpy()[i][:200])
    print("Label:", label_batch.numpy()[i], '\n')

Hasta este momento los pasos seguidos para la generación del dataset han sido comunes. A partir de ahora, durante este apartado, realizaremos una comparativa entre paso a paso.

De cara al **perceptrón**, dado que ya se exploraron las distintas posibilidades de vectorización, así que para este ejercicio vamos a elegir únicamente una de ellas. Vamos a elegir una **vectorización binaria**, donde para cada review, tendremos un vector de tamaño `VOCAB_SIZE`, que nos indicará si cada una de las palabras del vocabulario estaban presentes o no en el texto. Nos referiremos a las variables respectivas a este modelo con el prefijo `binary_*`.

Respecto al **modelo convolucional**, transformaremos el texto en una secuencia de índices de palabras, con el propósito de incluir un **embedding** a continuación. Se podría llegar a entrenar un modleo directamente utilizando los índices (sin utilizar embedding), pero dado que el índice del vocabulario no guarda ingún tipo de relación con el contenido semántico de las palabras (únicamente se tiene en cuenta su frecuencia), los resultados serían previsiblemente peores dado el ruido introducido. En ambos procesos utilizaremos el mismo tamaño de vocabulario `VOCAB_SIZE`. Nos referiremos a las variables respectivas a este modelo con el prefijo `seq_*`.

In [ ]:
binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [ ]:
seq_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

Generamos un dataset sin etiquetas para ajustar los vectorizadores, y los ajustamos (selección de vocabularios, generación de índices, etc).

**Nota:** Estos vectorizadores podían incluirse dentro de los modelos, pero por el momento los mantendremos fuera, para facilitar la exploración de los modelos y vectorizaciones, y también por términos de eficiencia computacional.

In [ ]:
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
seq_vectorize_layer.adapt(train_text)

Creamos dos funciones para aplicar la vectorización en los datasets:

In [ ]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [ ]:
def seq_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return seq_vectorize_layer(text), label

A continuación comparamos los resultados de ambos procesos de vectorización:

In [ ]:
# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review: ", first_review.numpy()[:200])
print("Label: ", first_label.numpy())

#### Ejemplo de vectorización binaria

In [ ]:
print("Vectorización binaria:")
print(binary_vectorize_text(first_review, first_label)[0])

#### Ejemplo de vectorización secuencial

In [ ]:
print("Vectorización secuencial:")
print(seq_vectorize_text(first_review, first_label)[0])

En el caso de la vectorización secuencial, podemos recuperar el texto original (salvo palabras fuera del vocabulario), dado que seguimos manteniendo el orden del mismo:

In [ ]:
print("2 ---> ", seq_vectorize_layer.get_vocabulary()[2])
print("3 ---> ", seq_vectorize_layer.get_vocabulary()[3])
print("Vocabulary size: {}".format(len(seq_vectorize_layer.get_vocabulary())))

In [ ]:
print("Reverse vectorization:")
for word_index in seq_vectorize_text(first_review, first_label)[0].numpy()[0]:
  print(seq_vectorize_layer.get_vocabulary()[word_index], end=' ')

### Preparación de datasets

En las siguientes celdas de código vamos a, finalmente incluir las vectorizazaciones en los datasets, y a hacer ajustes para paralelizar la vectorización en equipos con varios cores.

In [ ]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

seq_train_ds = raw_train_ds.map(seq_vectorize_text)
seq_test_ds = raw_test_ds.map(seq_vectorize_text)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_test_ds = configure_dataset(binary_test_ds)

seq_train_ds = configure_dataset(seq_train_ds)
seq_test_ds = configure_dataset(seq_test_ds)

### Modelos

A continuación generaremos, entrenaremos y evaluaremos ambos modelos.

**Nota**: A la hora de modelizar el problema, por las características del mismo, podríamos hacerlo utilizando modelos con un output bidimensional (una para la categoría "positive" y otro para la categoría negative), o un único output unidimensional que nos indique si es positivo o negativo. En nuestro caso, una opción un otra no debería representar una diferencia grande en términos de precisión o explicabilidad de los modelos. Las únicas dos consideraciones en este punto serían:
* Un output binario implicaría un **mayor número de parametros** a ajustar. Es posible que internamente se replicasen pesos con polaridades opuestas.
* La función de pérdida elegida sería distinta. `SparseCategoricalCrossentropy` o `CategoricalCrossentropy` cuando se enfoca com un problema multiclase y `BinaryCrossentropy` cuando se enfoca como un problema de regresión. 

#### Red simple (perceptron monocapa)

Vamos a crear u perceptrón monocapa para resolver un problema de dos clases.

In [ ]:
def create_simple_network():
  model = tf.keras.Sequential([layers.Dense(2)])
  model.compile(
      loss=losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer='adam',
      metrics=['accuracy'])
  return model

In [ ]:
simple_network = create_simple_network()
history = simple_network.fit(
    binary_train_ds, validation_data=binary_test_ds, epochs=5)

In [ ]:
simple_network.summary()

#### Red convolutiva

Vamos a entrenar una red convolutiva simple con un una única capa convolutiva, de 64 filtros.

El objetivo de esta red es simplemte comparar los primeros resultados. Después entreremos más en detalle.

In [ ]:
def create_simple_conv_model(vocab_size, num_labels):
  if num_labels == 1:
    loss = losses.BinaryCrossentropy(from_logits=True)
  else:
    loss = losses.SparseCategoricalCrossentropy(from_logits=True)
  
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  model.compile(
    loss=loss,
    optimizer='adam',
    metrics=['accuracy'])
  return model

In [ ]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
conv_model = create_simple_conv_model(vocab_size=VOCAB_SIZE + 1, num_labels=2)
conv_model.summary()

In [ ]:
history = conv_model.fit(seq_train_ds, validation_data=seq_test_ds, epochs=5)

Vamos a repetir este proceso, pero para un modelo con una única clasae, para confimar que no hay grandes diferencias en cuanto a la precisión del mismo, como anticipábamos previamente

In [ ]:
conv_model = create_simple_conv_model(vocab_size=VOCAB_SIZE + 1, num_labels=1)
conv_model.summary()

Como vemos, el número de parámetros de la última capa en este caso queda reducido a la mitad.

In [ ]:
history = conv_model.fit(seq_train_ds, validation_data=seq_test_ds, epochs=5)

Podemos observar que entre las dos versiones de los modelos convolucionales que hemos entrenado no hay apenas diferencias (ambos moelos se quedan en un 83% de precisión). Sin embargo, la fiferencia es notable a la hora de comparar con un modelo más sencillo, como es el perceptrón monocapa (87% de precisión).

Dejando a un lado variables como el tamaño del dataset número de epochs, se aprecia que los modelos convolucionales muestran un claro overfitting:
* perceptrón multicapa: `train acc 96% -  test acc 87%`
* rec convolucional: `train acc 100% - test acc 83%`

Algo que encaja a la perfección con el hecho de que el segundo modelo tenga un número considerablemente mayor de parámetros (unas 30 veces mayor).

### Exploración

Vamos a realizar una exploración rápida de ambos modelos, con el fin de establecer un pequeño paralelismo que sirva como introducción al siguiente apartado.



Si atendemos a los pesos que los modelos usan para decidir la categoría del texto, en el primer caso tenemos los pesos asociados directamente a la variable binaria que indica si una palabra está presente en el texto o no.

Si atendemos a las palabras cuyas apariciones son más relevantes para la clasificación, es relatiamente sencillo entender el las decisiones que toma el modelo.

Palabras con mayor peso para la categoría negativa:

In [ ]:
simple_network_weights = {k:v for k,v in zip(
    binary_vectorize_layer.get_vocabulary(), simple_network.layers[0].get_weights()[0])}

In [ ]:
sorted_simple_network_weights = {k: v for k, v in sorted(
    simple_network_weights.items(), key=lambda item: item[1][0], reverse=True)}
for k, v in list(sorted_simple_network_weights.items())[:10]:
  print(k, v[0])

Palabras con mayor peso para la categoría positiva:

In [ ]:
sorted_simple_network_weights = {k: v for k, v in sorted(
    simple_network_weights.items(), key=lambda item: item[1][1], reverse=True)}
for k, v in list(sorted_simple_network_weights.items())[:10]:
  print(k, v[1])

Si atendemos ahora al modelo convolutivo, vemos que las decisiones se toman en base a la activación o no de 64 filtros a lo largo del texto.

In [ ]:
conv_model.layers

In [ ]:
conv_model.layers[-1].get_weights()[0].shape

Entender la capa densa del clasificador es inmediato, una vez que se entiende el funcionamiento de los filtros previos, y los mapas de caracteríasticas que generan.

## Exploración detallada.

En esta sección vamos a profundizar en la clave de los modelos convolutivos: los filtros y los mapas de características, que no son más que mapas vectoriales la distribución de la activación de los filtros.

Como primer paso, vamos a generar un promer modelo que nos sirva de guía para la exploración del mismo. Dado que los modelos anteriores mostraban cierto overfitting, introduciremos capas de Dropuot como primera medida para reducirlo.

### Definición del primer modelo

In [ ]:
## Model Graph
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
emb = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
emb_drop = layers.Dropout(0.5)(emb)

# Conv1D + global max pooling
fmaps = layers.Conv1D(32, 5, padding="valid", activation="relu", strides=1)(emb_drop)
pfmaps = layers.GlobalMaxPooling1D()(fmaps)

# We add a vanilla hidden layer:
classifier = layers.Dense(32, activation="relu")(pfmaps)
classifier_drop = layers.Dropout(0.5)(classifier)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(classifier_drop)

In [ ]:
## Model build
conv_model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
conv_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
conv_model.fit(seq_train_ds, validation_data=seq_test_ds, epochs=5)

### Modelo de activación

Partiendo del modelo que acabamos de entrenar, vamos a generar un modelo cuyo output sean los mapas de activación de los distintos filtros.


In [ ]:
def gen_activation_model(model):
  layer_outputs = []
  layer_names = []
  for layer in conv_model.layers:
      if isinstance(layer, (tf.keras.layers.Conv1D, tf.keras.layers.GlobalMaxPooling1D)):
          layer_outputs.append(layer.output)
          layer_names.append(layer.name)
  activation_model = tf.keras.Model(inputs=model.inputs, outputs=layer_outputs)
  return layer_names, activation_model

In [ ]:
layer_names, activation_model = gen_activation_model(conv_model)

Vamos también a crear una frase de ejemplo que tenga patrones positivos y negativos en una misma frase, que nos sirva para estudiar los distintos patrones de activación.

In [ ]:
good_bad_review = "Last Thursday I went to the movies to see Tarantino's latest film. \
    I thought the beginning of the movie was horrible, \
    and a complete waste of time. Boring, the plot was poor and disappointing. \
    However, everything changed after the first 15 minutes. The rest of the movie was \
    perfect, with a brilliant plot and great performances."

In [ ]:
t = TextPreprocessor('lemma')
good_bad_review = t.transform([good_bad_review])[0]

In [ ]:
sample_vecs = seq_vectorize_layer([good_bad_review])
feature_maps, pooled_feature_maps = activation_model.predict(sample_vecs)

Dado que nuestro modelo tenía 32 filtros, y una longitud máxima de 250 tokens, para un texto tendremos los 32 mapas de activación crrespondientes, consistentes en 246 evaluaciones (debido al tamaño del filtro, strides y padding elegido)

In [ ]:
feature_maps.shape

Posteriormente, se añade una capa GlobalMaxPooling1D, que por cada mapa de activación, nos devolverá su valor máximos. Básicamente esto nos indicará si se ha detectado un patrón concreto en el texto y su nivel de activación.

In [ ]:
pooled_feature_maps.shape

### Activación de un filtro específico

De este modo, disponemos de la vectorización del texto:

In [ ]:
sample_vecs[0]

Para el décimo filtro, por ejemplo, el modelo tiene una activación de:

In [ ]:
pooled_feature_maps[0][10]

Cuyo mápa de características ha sido, para este texo:

*únicamente mostramos la evaluación de las primeras 50 ventanas de activación:

In [ ]:
feature_maps[0,:,10][:50]

In [ ]:
vocab = seq_vectorize_layer.get_vocabulary()

In [ ]:
words = []
for word_index in sample_vecs[0].numpy():
  if word_index > 0:
    words.append(seq_vectorize_layer.get_vocabulary()[word_index])

In [ ]:
activations = []
for i in range(2, len(words)-2):
  window = ' '.join([words[i + j] for j in range(-2,3)])
  activations.append({'index': i, 'window':window, 'activation': feature_maps[0,i,10]})
activations = pd.DataFrame(activations).set_index('index')

In [ ]:
sns.lineplot(data=activations, x='window', y='activation', linewidth=2.5)
_ = plt.xticks(rotation = 'vertical')

Podemos ver cómo la activación tras aplicar este filtro es bastante alta cuando apaarecen expresiones positivas, mientras que cuando no es así permanece baja. De hecho, este filtro parece encargarse de detectar patrones principalmente positivos, dado que no hay activación con palabras claramente negativas.

### Patrones de activación

Ahora que hemos repasado en detalle una pequeña parte del modelo, vamos a intentar obtener una visión más global del mismo.

Para ello, en primer lugar vamos a intentar visualizar la activación de odos los filtros.

In [ ]:
def process_feature_maps(sample_vecs, feature_maps, vocab, n_filters=32):
  feature_maps_data = []
  for fmap_index in range(n_filters):
    #print('fmap', fmap_index)
    for i, token_index in enumerate(sample_vecs[0]):
      if token_index > 1:
        try: # cambiar esto para que itere sobre los fmaps
          #print(i, vocab[token_index.numpy()], sample_vecs[0][i].numpy(), feature_maps[0,i,fmap_index])
          feature_map_data = {}
          feature_map_data['filter'] = fmap_index
          feature_map_data['word'] = vocab[token_index.numpy()]
          feature_map_data['word_index'] = sample_vecs[0][i].numpy()
          feature_map_data['word_filter_weight'] = feature_maps[0,i,fmap_index]
          feature_maps_data.append(feature_map_data)
        except IndexError:
          pass
    print()
  feature_maps_data = pd.DataFrame(feature_maps_data)
  return feature_maps_data

In [ ]:
feature_maps_data = process_feature_maps(sample_vecs, feature_maps, vocab)

**Nota**: Para facilitar la visualización y dado que nuestra ventana es de 5 tokens, en el fráfico de activación vamo s amostrar únicamente la palabra palabra central de cada ventana en el ejehorizontal, para evitar grandes líneas de texto.

In [ ]:
ax = sns.lineplot(
    data=feature_maps_data,
    x='word',
    y='word_filter_weight',
    hue='filter',
    linewidth=2.5
)
_ = plt.xticks(rotation = 'vertical')

Como se puede observar, en una vista global del modelo, hay grandes activaciones cuando se dan expresiones claramente positivas y/o negativas.

Podemos observar que muchos filtros se activan en las mismas zonas, lo que podría indicar que el modelo está aprendiendo patrones redundantes.

#### Exploración de mapas de características

También podemo sobservar cómo hay filtros que no se activan o lo hanpoco . Para tener un mejor detalle de esto último, vamos a intentar generar histogramas de cada uno de los mapas de características.

In [ ]:
feature_maps_data['column'] = feature_maps_data['filter'] // 10
feature_maps_data['row'] = feature_maps_data['filter'] % 10

In [ ]:
sns.displot(
    feature_maps_data,
    x="word_filter_weight",
    col="column",
    row="row",
    binwidth=0.05,
    binrange=(0, 1)
)

Podemos ver que hay histogramas que reflejan muchos valores cercanos a cero. No es necesariamente un problemadado que únicamente estamos viendo una frase concreta, y no un dataset completo, pero podría ser un indicador de que el modelo está manejando muchos más parámetros de los necesarios.

A través de una exploración manual, sí podemos encontrar algunos filtros que aprenden a detectar patrones irrelevantes para el problema.

In [ ]:
feature_maps_data[feature_maps_data['filter'] == 0]

#### Aprendizaje de filtros

Vamos a revisar algunos histogramas de los pesos de los filtros de la capa convolutiva, para detectar si existen filtros que no están aprendiendo.

In [ ]:
conv_model.layers[3].get_weights()[0].shape

In [ ]:
filter_data = []
filters = conv_model.layers[3].get_weights()[0]
for token in range(5):
  for dim in range(64):
    for i, weight in enumerate(filters[token][dim]):
      filter_data.append(
          {
              'token': token,
              'dim': dim,
              'filter': i,
              'weight': weight
          }
      )
filter_data = pd.DataFrame(filter_data)

In [ ]:
filter_data

In [ ]:
filter_data['column'] = filter_data['filter'] // 10
filter_data['row'] = filter_data['filter'] % 10

In [ ]:
sns.displot(
    filter_data,
    x="weight",
    col="column",
    row="row",
    binwidth=0.05,
    binrange=(0, 1)
)

Podemos observar que en este punto no existen filtros cuyos pesos sean todos muy cercanos a cero. Este problema suele ser común en modelos que trabajan con imágenes, donde se concatenan varias capas convolucionales.

### Heatmap

### Embeddings

Un componenete fundamental en este model es el embbeding que se utiliza para vectorizar los textos. Vamos a hacer una primera exploración para comprobar que tiene sentido.

Dado que intentar explorar todo el vocabulario es un porblema complejo  que daría para una práctica en sí misma, vamos a intentar centrarnos en el ejemplo con el que venimos trabajando.

In [ ]:
seq_vocab = seq_vectorize_layer.get_vocabulary()
word_embeddings = conv_model.layers[1].get_weights()[0]

Para facilitar la visualización de los embeddings, vamos a utilizar TSNE para reducir la dimensionalidad. Adicionalmente, mostraremos únicamente las palabras del texto de ejemplo.

In [ ]:
word_embeddings_2 = TSNE(
    n_components=2,
    learning_rate='auto',
    init='random',
    perplexity=30).fit_transform(word_embeddings)

In [ ]:
word_embeddings_2.shape

In [ ]:
df = pd.DataFrame(word_embeddings_2, columns=['x', 'y'])
# adding a columns for the corresponding words
df['words'] = seq_vocab
df['in_sentence'] = df['words'].isin(feature_maps_data['word'].tolist())

In [ ]:
df = df[df['in_sentence']]

In [ ]:
# plotting a scatter plot
fig = px.scatter(df, x="x", y="y", text="words")
# adjusting the text position
fig.update_traces(textposition='top center')
# setting up the height and title
fig.update_layout(
    height=600,
    title_text='Word embedding chart'
)

Podemos observar como hay regiones donde se sitúan todas las palabras negativas, y otras donde se acumulan las positivas, que es el comportamiento esperado.

## Pruebas

Ahora que ya hemos entendido en detalle cómo funcionnan los modelos convolucionales aplicados a texto, vamos a llevar a cabo una serie de pruebas que nos ayuden a entender mejor el impacto de las distintas configuraciones del modelo.

### Generación de modelos

Vamos a a partir de un modelo similar al que hemos estado trabajando en la sección anterior, y vamos a comenzar a variar parámetros para entender su impacto.

Vamos a escribir una función que nos permita parametrizar diversos aspectos del model. Entre ellos, la capacidad de generar bloques de capas convolucionales, siendo cada bloque una concatenación de capas de este tipo seguidas de una capa de tipo Pooling.

In [ ]:
def gen_conv_model(n_blocks=1, n_conv_layers_per_block=1, filter_size=5, n_filters=32, optimizer="adam"):
  inputs = tf.keras.Input(shape=(None,), dtype="int64")
  emb = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
  emb_drop = layers.Dropout(0.5)(emb)

  for _ in range(n_blocks):
    for _ in range(n_conv_layers_per_block):
      fmaps = layers.Conv1D(
          n_filters,
          filter_size,
          padding="valid",
          activation="relu",
          strides=1)(emb_drop)
    pfmaps = layers.MaxPooling1D(pool_size=2)(fmaps)


  gpfmaps = layers.GlobalMaxPooling1D()(pfmaps)

  # We add a vanilla hidden layer:
  classifier = layers.Dense(32, activation="relu")(gpfmaps)
  classifier_drop = layers.Dropout(0.5)(classifier)

  # We project onto a single unit output layer, and squash it with a sigmoid:
  predictions = layers.Dense(1, activation="sigmoid", name="predictions")(classifier_drop)

  ## Model build
  conv_model = tf.keras.Model(inputs, predictions)

  # Compile the model with binary crossentropy loss and an adam optimizer.
  conv_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

  return conv_model

Vamos a generar un par de callbacks que pueden ayudarnos en los entrenamientos.

In [ ]:
learning_rate_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    min_lr=0.001
)

early_stopping_cb = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=5,
    min_delta=0.001
)

A la hora de entrenar los modelos, vamos a fijar el número de epocs por el momento, eliminando el callback de `early_stopping`.

In [ ]:
def train_conv_model(model):
  return model.fit(
      seq_train_ds,
      validation_data=seq_test_ds,
      epochs=10,
      callbacks=[
          learning_rate_cb,
          #early_stopping_cb
      ],
      verbose = 0
  )

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

### Tamaño del filtro

En primer lugar vamos a intentar ver el impacto que tiene el tamaño del filtro en el modelo. Comenzaremos con una única capa `Conv1D`, y variaremos su tamaño.

In [ ]:
tests = range(1,15)

In [ ]:
results = []
for i, filter_size in enumerate(tests):
  tf.keras.backend.clear_session()
  print('Testing filter size:', filter_size)
  model = gen_conv_model(filter_size=filter_size)
  hist = train_conv_model(model)
  acc = max(hist.history['val_accuracy'])
  epoch = max(hist.epoch)
  results.append(
      {
        'test_number': i,
        'acc': acc,
        'epoch': epoch,
        'filter_size': filter_size
      }
  )
  del model

In [ ]:
filter_size_results_df = pd.DataFrame(results)
ax1 = sns.lineplot(data=filter_size_results_df, x="filter_size", y="acc", color="blue")
ax2 = plt.twinx()
ax2 = sns.lineplot(data=filter_size_results_df, x="filter_size", y="epoch", color="red", ax=ax2)

Al parecer por las pruebas, existe una tendencia a mejorar el rendimiento del modelo al aumentar el tamaño del filtro. Es cierto que hay un pico con filtros de tamaño 2, y habría que realizar más pruebas para ver si es un caso aislado, o para ese tamaño de filtro existe un aumento real de rendimiento.

### Número de filtros

A continuación vamos a variar el número de filtros, y medir su impacto.

In [ ]:
tests = range(1,250,10)

In [ ]:
results = []
for i, n_filters in enumerate(tests):
  tf.keras.backend.clear_session()
  print('Testing n_filters:', n_filters)
  model = gen_conv_model(filter_size=5, n_filters=n_filters)
  hist = train_conv_model(model)
  acc = max(hist.history['val_accuracy'])
  epoch = max(hist.epoch)
  results.append(
      {
        'test_number': i,
        'acc': acc,
        'epoch': epoch,
        'n_filters': n_filters
      }
  )

In [ ]:
n_filters_results_df = pd.DataFrame(results)
ax1 = sns.lineplot(data=n_filters_results_df, x="n_filters", color="blue", y="acc")
ax2 = plt.twinx()
ax2 = sns.lineplot(data=n_filters_results_df, x="n_filters", color="red", y="epoch", ax=ax2)

A partir de 30-40 filtros vemos cómo el rendimiento no aumenta al aumentar el número de filtros. Para el problema que estamos resolviendo, y un dataset contenido, parece que una mayor cantidad de filtros no aporta gran cosa. Previamente ya habíamos "sospechado" que para 32 filtros ya podían existir filtros poco relevantes para la clasificación.

### Número de capas convolucionales en un bloque

En este caso vamos a generar un único bloque de capas convolucionales, y exploraremos qué ocurre cuando éste bloque aumenta en profundidad.

In [ ]:
tests = range(1,5)

In [ ]:
results = []
for i, n_conv_layers_per_block in enumerate(tests):
  tf.keras.backend.clear_session()
  print('Testing n_conv_layers_per_block:', n_conv_layers_per_block)
  model = gen_conv_model(
      filter_size=15,
      n_filters=32,
      n_conv_layers_per_block=n_conv_layers_per_block
  )
  hist = train_conv_model(model)
  acc = max(hist.history['val_accuracy'])
  epoch = max(hist.epoch)
  results.append(
      {
        'test_number': i,
        'acc': acc,
        'epoch': epoch,
        'n_conv_layers_per_block': n_conv_layers_per_block
      }
  )

In [ ]:
n_conv_layers_per_block_results_df = pd.DataFrame(results)
ax1 = sns.lineplot(data=n_conv_layers_per_block_results_df, x="n_conv_layers_per_block", color="blue", y="acc")
ax2 = plt.twinx()
ax2 = sns.lineplot(data=n_conv_layers_per_block_results_df, x="n_conv_layers_per_block", color="red", y="epoch", ax=ax2)

Aunque el rango de pruebas no ha sido demaisado grande, se aprecia una clara caída con el aumento en profundidad del bloque. Dado que hemos fijado el número de epochs, es posible que con mayor profundidad se requieran entrenamientos más largos. 

Para descartar esto, vamos a realizar un entrenamiento aislado.

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = gen_conv_model(
    filter_size=15,
    n_filters=32,
    n_conv_layers_per_block=4
)

In [ ]:
model.fit(
    seq_train_ds,
    validation_data=seq_test_ds,
    epochs=20,
    callbacks=[
        learning_rate_cb,
        early_stopping_cb
    ]
)

In [ ]:
plot_history(history)

Tras este entrenamiento se aprecia el fenómeno de overfitting claro, y un detrimento de la precisión al avanzar en el entrenamiento, por lo tanto descartamos nuestra hipótesis anterior.

### Número de bloques

Veamos ahora qué ocurre cuando aumentamos el número de bloques que introducimos en el modelo.

In [ ]:
tests = range(1,10)

In [ ]:
results = []
for i, n_blocks in enumerate(tests):
  tf.keras.backend.clear_session()
  print('Testing n_blocks:', n_blocks)
  model = gen_conv_model(
      filter_size=15,
      n_filters=32,
      n_conv_layers_per_block=1,
      n_blocks=n_blocks
  )
  hist = train_conv_model(model)
  acc = max(hist.history['val_accuracy'])
  epoch = max(hist.epoch)
  results.append(
      {
        'test_number': i,
        'acc': acc,
        'epoch': epoch,
        'n_blocks': n_blocks
      }
  )

In [ ]:
n_blocks_results_df = pd.DataFrame(results)
ax1 = sns.lineplot(data=n_blocks_results_df, x="n_blocks", y="acc", color="blue")
ax2 = plt.twinx()
ax2 = sns.lineplot(data=n_blocks_results_df, x="n_blocks", y="epoch", color="red", ax=ax2)

Entre 1 y 4 bloques hemos obtenido un rendimiento creciente. Sin embargo, a partir de ese número de bloques los resultados de la progresión en el número de filtros se vuelve errática. Dada la complejidad de modelos con esa profundidad, es difícil dar una explicación en este momento, y requeriría una serie de pruebas específicas.

Por el momento tomamos 4 como un buen número de bloques.

### Embeddings pre-entrenados

Hasta el momento todas nuestras pruebas se han llevado a cabo entrenando embeddings desde cero para este problema. 

Vamos a ver el impacto que tiene utilizar embeddings pre-entrenados.

En primer lugar entrenamos un modelo sencillo utilzando embeddings entrenados desde cero, como hemos hecho hasta ahora, y después replicaremos el problema con una vectorización pre-ajustada.

In [ ]:
conv_model = create_simple_conv_model(vocab_size=VOCAB_SIZE + 1, num_labels=2)
conv_model.summary()

In [ ]:
history = conv_model.fit(seq_train_ds, validation_data=seq_test_ds, epochs=10)

In [ ]:
plot_history(history)

En este punto hemos obtenido una precisión cercana al 84% para este problema.

Vamos a utilizar un embedding de GloVe con 50 dimensiones (es el más cercano a las 64 que hemos estado usando), congelaremos la capa de Embedding para que no sufra cambios con el entrenamiento. Esto nos dará un modelo con menos parámetros, y por tanto más estable de cara al entrenamiento.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

Cargamos los vectores:

In [ ]:
path_to_glove_file = os.path.join(
    "glove.6B.50d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
voc = seq_vectorize_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

Generamos ahora la matriz de embeddings

In [ ]:
num_tokens = VOCAB_SIZE + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


In [ ]:
embedding_matrix.shape

Y creamos un modelo como el que utilizamos en el entrenamiento anterior, pero cargando los pesos del nuvo embedding.

In [ ]:
model = tf.keras.Sequential([
    layers.Embedding(
        VOCAB_SIZE + 2,
        50,
        embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        trainable=False
    ),
    layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
    layers.GlobalMaxPooling1D(),
    layers.Dense(2)
])
model.compile(
  loss=losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer='adam',
  metrics=['accuracy'])

In [ ]:
history = model.fit(seq_train_ds, validation_data=seq_test_ds, epochs=10)

In [ ]:
plot_history(history)

Podemos ver como los resultados obtenidos son notablemente inferiores al modelo en el que se entrenó esta capa como parte del modelo (un 10% de caída en precisión).

Esto en parte se debe a que esta capa no ha sido entrenada con el fin de resolver este problema, por lo que nuestro siguiente paso será replicar este entrenamiento, pero sin congelar la capa de embedding, de manera que puedan modificarse sus valores durante el entrenamiento.

In [ ]:
model = tf.keras.Sequential([
    layers.Embedding(
        VOCAB_SIZE + 2,
        50,
        embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        trainable=True
    ),
    layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
    layers.GlobalMaxPooling1D(),
    layers.Dense(2)
])
model.compile(
  loss=losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer='adam',
  metrics=['accuracy'])

In [ ]:
history = model.fit(seq_train_ds, validation_data=seq_test_ds, epochs=10)

In [ ]:
plot_history(history)

Podemos observar cómo hay una mejora de la precisión (sobrepasamos el 80%), pero sin llegar a los resultado previos.

Entrenar un modelo con valores reentrenados es un proceso complejo, pues para modificar el embedding preentrenado deberíamos usar optimizadores con learning rates reducidos, mientras que el resto del modelo requiere un tratamiento más agresivo.

Sin duda podríamos llegar a la misma precisión que utilizando un embedding entrenado desde cero, pero por el momento parece que introduciría complejidad al entrenamiento sin obtener grandes beneficios.

### Cambios en el tratamiento del texto

Hasta el momento no hemos prestado atención al tipo de procesado que sufría el texto, así que vamos ha ver cómo impacta en los resultados.

Como en la práctica anterior se profundizó en este tema, no vamos a dar excesivos detalles. Probaremos los tres niveles de pre procsado del texto que vimos al inicio de esta misma práctica.

In [ ]:
tests = ['basic', 'lemma', 'stem']

for t in tests:
  print(f"Training with {t} processing level...")
  tf.keras.backend.clear_session()
  raw_train_ds = tf.data.Dataset.from_tensor_slices(
      (train_df[f'{t}_review'], train_df['label'].values)).batch(BATCH_SIZE)
  raw_test_ds = tf.data.Dataset.from_tensor_slices(
      (test_df[f'{t}_review'], test_df['label'].values)).batch(BATCH_SIZE)

  seq_vectorize_layer = TextVectorization(
      max_tokens=VOCAB_SIZE,
      output_mode='int',
      output_sequence_length=MAX_SEQUENCE_LENGTH)

  train_text = raw_train_ds.map(lambda text, labels: text)
  seq_vectorize_layer.adapt(train_text)

  seq_train_ds = raw_train_ds.map(seq_vectorize_text)
  seq_test_ds = raw_test_ds.map(seq_vectorize_text)

  seq_train_ds = configure_dataset(seq_train_ds)
  seq_test_ds = configure_dataset(seq_test_ds)

  conv_model = gen_conv_model(
      filter_size=15,
      n_filters=32,
      n_conv_layers_per_block=1,
      n_blocks=4
  )

  history = conv_model.fit(
      seq_train_ds, validation_data=seq_test_ds, epochs=5, verbose=0)

  plot_history(history)
  print("Test acc:", history.history['val_accuracy'][-1], "\n")

Podemos ver cómo la lematización y el stemming aportan un extra respecto al pre-procesado básico, probablemente debido a la unificación de términos con carga semántica casi similar, que implica una mayor condensación de la información.

Entre la lematización y el stemming, los resultado son muy similares, estando la lematización levemente mejor posicionada en nuestras pruebas. Sin embargo, esa diferencia podría no ser significativa dada la simplicidad de las pruebas.

Por otra parte, el stemming es limitante con frecuencia a la hora de utilizar embeddings pre entrenados.

### Algoritmos de optimización

Llevaremos a cabo una comparativa en cuanto a los algoritmos de optimización, no tanto para encontrar el mejor de ellos (aquí se requeriría un trabajo importante de meta optimización de parámetros de cada uno de ellos), si no para entender si hay grandes diferencias entre unos y otros con este tipo de modelos.

Dados los resultados del apartado anterior, vamos a partir del dataset con lematización aplicada.

In [ ]:
raw_train_ds = tf.data.Dataset.from_tensor_slices(
      (train_df[f'lemma_review'], train_df['label'].values)).batch(BATCH_SIZE)
raw_test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df[f'lemma_review'], test_df['label'].values)).batch(BATCH_SIZE)

seq_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

train_text = raw_train_ds.map(lambda text, labels: text)
seq_vectorize_layer.adapt(train_text)

seq_train_ds = raw_train_ds.map(seq_vectorize_text)
seq_test_ds = raw_test_ds.map(seq_vectorize_text)

seq_train_ds = configure_dataset(seq_train_ds)
seq_test_ds = configure_dataset(seq_test_ds)

In [ ]:
for optimizer in ["adam", "adagrad", "nadam", "ftrl", "nadam"]:
  print("Optimizer:", optimizer)
  tf.keras.backend.clear_session()
  ## Model build
  conv_model = gen_conv_model(
      filter_size=15,
      n_filters=32,
      n_conv_layers_per_block=1,
      n_blocks=4
  )

  # Compile the model with binary crossentropy loss and an adam optimizer.
  conv_model.compile(loss="binary_crossentropy",
                    optimizer=optimizer,
                    metrics=["accuracy"])

  history = conv_model.fit(
        seq_train_ds,
        validation_data=seq_test_ds,
        epochs=20,
        callbacks=[
            learning_rate_cb,
            early_stopping_cb
        ],
        verbose=0
  )

  print("Test acc:", history.history['val_accuracy'][-1], "\n")
  plot_history(history)

Como vemos, hay grandes diferencias entre algoritmos de optimización, pudiendo lastrar mucho el aprendizaje del modelo si la elección no es correcta. 

Entre aquellos que ofrecen un mejor rendimiento, como se ha comentado, faltaría realizar un trabajo de meta optimización de parámetros específicos de esos algoritmos para optener los mejores resultados. Consideramos que esto está fuera del alcance de esta práctico.

## Conclusión

En esta práctica hemos repasado modelos que ofrecen un paso más en capacidad de abstracción y complejidad sobre lo que se vió en la práctica anterior. Previamente trabajábamos con la presencia o no de palabras en un texto, y trabajamos con la presencia o no de ciertos patrones, no palabras aisladas. Esto es un aspecto fundamental para el tratamiento de textos, dado que no es lo mismo "it is a good movie" que "it is **not** a good movie".

Sin embargo, este avance en capacidades del modelo va ligado a una mayor complejidad, y require un mayor conocimiento de los modelos para poder entender qué está ocurriendo. Sin embargo, como hemos visto que la explicabilidad de estos modelos, una vez que se entienden sus mecánicas, es posible.

De entre todos los factores existentes a la hora de desarrollar un modelo, entender el propio modelo es fundamental para poder mejorarlo. Por lo tanto considero que esta práctica es tremendamente útil.